# Colonies

Data Source: http://www.cepii.fr/

In [1]:
import pandas as pd
country_info = pd.read_excel("../../data/geo_cepii.xls")

In [2]:
country_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   iso2              237 non-null    object 
 1   iso3              238 non-null    object 
 2   cnum              238 non-null    int64  
 3   country           238 non-null    object 
 4   pays              238 non-null    object 
 5   area              238 non-null    int64  
 6   dis_int           238 non-null    float64
 7   landlocked        238 non-null    int64  
 8   continent         238 non-null    object 
 9   city_en           238 non-null    object 
 10  city_fr           238 non-null    object 
 11  lat               238 non-null    float64
 12  lon               238 non-null    float64
 13  cap               238 non-null    int64  
 14  maincity          238 non-null    int64  
 15  citynum           238 non-null    object 
 16  langoff_1         238 non-null    object 
 1

Some of the columns are objects and end up being mixed type.. this is not really playing well with sql.. 

In [3]:
country_col = country_info[["iso3","colonizer1","colonizer2","colonizer3","colonizer4","short_colonizer1","short_colonizer2","short_colonizer3"]]

This doesn't work in jupyter lab.. just in jupyter notebook

In [4]:
from fugue_notebook import setup
import fugue_duckdb

setup()

<IPython.core.display.Javascript object>

In [5]:
%%fsql
SELECT * FROM country_col


PRINT

ANTLR runtime and generated code versions disagree: 4.11.1!=4.10.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.10.1


,iso3,colonizer1,colonizer2,colonizer3,colonizer4,short_colonizer1,short_colonizer2,short_colonizer3
0,ABW,NLD,.,.,.,.,.,.
1,AFG,.,.,.,.,GBR,.,.
2,AGO,PRT,.,.,.,.,.,.
3,AIA,GBR,.,.,.,.,.,.
4,ALB,TUR,.,.,.,.,.,.
5,AND,.,.,.,.,.,.,.
6,ANT,NLD,.,.,.,.,.,.
7,ARE,GBR,.,.,.,.,.,.
8,ARG,ESP,.,.,.,.,.,.
9,ARM,TUR,RUS,.,.,.,.,.


Took a while to debug.. forgot the group by and the query still worked but wo an error and with funny results

In [6]:
%%fsql
SELECT colonies.*
, count_col1 + count_col2 + count_col3 + count_col4 AS total_col FROM
(SELECT c.iso3
    , count(c1.iso3) AS count_col1
    , count(c2.iso3) AS count_col2
    , count(c3.iso3) AS count_col3
    , count(c4.iso3) AS count_col4
    , count(sc1.iso3) AS count_scol1
    , count(sc2.iso3) AS count_scol2
    , count(sc3.iso3) AS count_scol3
    , CASE WHEN (c.colonizer1 == 'None' AND c.short_colonizer1 == 'None') THEN 1 ELSE 0 END AS FREE
FROM country_col AS c
LEFT JOIN country_col AS c1 ON c1.colonizer1 = c.iso3
LEFT JOIN country_col AS c2 ON c2.colonizer2 = c.iso3
LEFT JOIN country_col AS c3 ON c3.colonizer3 = c.iso3
LEFT JOIN country_col AS c4 ON c4.colonizer4 = c.iso3
LEFT JOIN country_col AS sc1 ON sc1.short_colonizer1 = c.iso3
LEFT JOIN country_col AS sc2 ON sc2.short_colonizer2 = c.iso3
LEFT JOIN country_col AS sc3 ON sc3.short_colonizer3 = c.iso3
GROUP BY c.iso3) AS colonies
YIELD DATAFRAME AS colonies
SAVE OVERWRITE "/tmp/colonies.csv" (header=true)

ANTLR runtime and generated code versions disagree: 4.11.1!=4.10.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.10.1


In [7]:
!mv /tmp/colonies.csv ../dataset/